In [1]:
import numpy as np
import queue
from functools import reduce

In [49]:
def make_chains(mat, names, k, key):
    
    def dfs(que, mat, v, k_now, k_max, chainsDict, names):
        
        if k_now == k_max:
            print(chainsDict)
            name_in_num = list(que)
            name_in_num.sort()
            name_in_num = list(map(lambda i: names[i], name_in_num))
            kk = reduce(lambda i, j: i + j, name_in_num)
            chainsDict.setdefault(kk, 0)
            print(chainsDict)
            chainsDict[kk] += 1
            return
        
        
        for i in range(0, mat.shape[0]):
            if mat[v, i] == 1 and not i in que:
                que.append(i)
                dfs(que, mat, i, k_now + 1, k_max, chainsDict, names)
                del(que[k_now])
                
            
    chainsDict = dict.fromkeys(key, 0)
    
    for i in range(mat.shape[0]):#Запускаем обход в глубину k из каждой вершины
        que = []
        que.append(i)
        
        dfs(que, mat, i, 1, k, chainsDict, names)
        del(que[0])
        
        
    for i in chainsDict.keys(): #Каждую цепочку посчитали два раза
        chainsDict[i] //= 2
    return chainsDict
        

In [50]:
def getAllKChainsNameInFile(filename, k): #k-размер цепочек
    all_chains_names = []
    ctr = 0
    with  open(filename, "r") as f:
        
        
        for i in f:
            ctr += 1
            mol_atoms = []
            mol_name = i.split()[0]
            num_elems = int(i.split()[1])
            

            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                line = next(f) #Берём i линию в молекуле
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "000":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, {})
            all_chains_names += res.keys()
          
    return all_chains_names, ctr

In [51]:
filename = "GLASS.STR"

In [52]:
f = open(filename, "r")

In [53]:
names = getAllKChainsNameInFile(filename, 3)

{}
{'1C2C3C': 0}
{'1C2C3C': 1}
{'1C2C3C': 1, '2C3C4C': 0}
{'1C2C3C': 1, '2C3C4C': 1}
{'1C2C3C': 1, '2C3C4C': 1}
{'1C2C3C': 2, '2C3C4C': 1}
{'1C2C3C': 2, '2C3C4C': 1, '3C4C5C': 0}
{'1C2C3C': 2, '2C3C4C': 1, '3C4C5C': 1}
{'1C2C3C': 2, '2C3C4C': 1, '3C4C5C': 1, '3C4C6C': 0}
{'1C2C3C': 2, '2C3C4C': 1, '3C4C5C': 1, '3C4C6C': 1}
{'1C2C3C': 2, '2C3C4C': 1, '3C4C5C': 1, '3C4C6C': 1}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 1, '3C4C6C': 1}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 1, '3C4C6C': 1}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C6C': 1}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C6C': 1, '4C5C6C': 0}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C6C': 1, '4C5C6C': 1}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C6C': 1, '4C5C6C': 1}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C6C': 2, '4C5C6C': 1}
{'1C2C3C': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C6C': 2, '4C5C6C': 1}
{}
{'1C2C3C': 0}
{'1C2C3C': 1}
{'1C2C3C': 1, '2C3C4C': 0}
{'1C2C3C': 1, '2C3C4C': 1}
{'1C2C3C': 1, '2C3C4C': 1}
{'1C2C3C': 2, '2C3C4C

{'1C2C3O': 2, '1C2C4C': 1, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1}
{'1C2C3O': 2, '1C2C4C': 1, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1, '2C3O4C': 0}
{'1C2C3O': 2, '1C2C4C': 1, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1, '2C3O4C': 1}
{'1C2C3O': 2, '1C2C4C': 1, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1, '2C3O4C': 1}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1, '2C3O4C': 1}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1, '2C3O4C': 1}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1, '2C3O4C': 2}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 1, '2C3O4C': 2}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 2, '2C3O4C': 2}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 1, '1C5C7C': 1, '1C2C5C': 2, '2C3O4C': 2}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 2, '1C5C7C': 1, '1C2C5C': 2, '2C3O4C': 2}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 2, '1C5C7C': 1, '1C2C5C': 2, '2C3O4C': 2, '5C6C7C': 0}
{'1C2C3O': 2, '1C2C4C': 2, '1C5C6C': 2, '1C5C7C': 1,

{'1C2C3C': 2, '1C2C4C': 2, '1C2C10C': 1, '2C4C5C': 2, '2C4C9C': 1, '2C10C11C': 1, '2C10C15C': 1, '2C3C4C': 2, '2C3C10C': 1, '2C4C10C': 1, '4C5C6C': 2, '4C8C9C': 1, '4C5C9C': 1, '5C6C7C': 1, '6C7C8C': 1, '6C7C16O': 1}
{'1C2C3C': 2, '1C2C4C': 2, '1C2C10C': 1, '2C4C5C': 2, '2C4C9C': 1, '2C10C11C': 1, '2C10C15C': 1, '2C3C4C': 2, '2C3C10C': 1, '2C4C10C': 1, '4C5C6C': 2, '4C8C9C': 1, '4C5C9C': 1, '5C6C7C': 2, '6C7C8C': 1, '6C7C16O': 1}
{'1C2C3C': 2, '1C2C4C': 2, '1C2C10C': 1, '2C4C5C': 2, '2C4C9C': 1, '2C10C11C': 1, '2C10C15C': 1, '2C3C4C': 2, '2C3C10C': 1, '2C4C10C': 1, '4C5C6C': 2, '4C8C9C': 1, '4C5C9C': 1, '5C6C7C': 2, '6C7C8C': 1, '6C7C16O': 1, '7C8C9C': 0}
{'1C2C3C': 2, '1C2C4C': 2, '1C2C10C': 1, '2C4C5C': 2, '2C4C9C': 1, '2C10C11C': 1, '2C10C15C': 1, '2C3C4C': 2, '2C3C10C': 1, '2C4C10C': 1, '4C5C6C': 2, '4C8C9C': 1, '4C5C9C': 1, '5C6C7C': 2, '6C7C8C': 1, '6C7C16O': 1, '7C8C9C': 1}
{'1C2C3C': 2, '1C2C4C': 2, '1C2C10C': 1, '2C4C5C': 2, '2C4C9C': 1, '2C10C11C': 1, '2C10C15C': 1, '2C3C4C':

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




{'1C2C3C': 2, '1C5C6C': 2, '1C13N15C': 2, '1C13N27C': 1, '1C2C6C': 2, '1C2C13N': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C7C': 2, '4C5C6C': 2, '4C7C8C': 2, '4C7C12C': 2, '4C5C7C': 2, '1C6C13N': 2, '7C8C9C': 2, '7C11C12C': 2, '7C8C12C': 2, '8C9C10C': 2, '9C10C11C': 2, '9C10C14N': 2, '10C11C12C': 2, '10C14N21C': 1, '10C14N37C': 1, '10C11C14N': 2, '13N15C16C': 2, '13N15C20C': 1, '13N27C28C': 1, '13N27C32C': 1, '14N21C22C': 1, '14N21C26C': 1, '14N37C38C': 1, '14N37C42C': 1, '13N15C27C': 1, '15C16C17C': 1, '15C19C20C': 1, '15C16C20C': 1, '16C17C18C': 0}
{'1C2C3C': 2, '1C5C6C': 2, '1C13N15C': 2, '1C13N27C': 1, '1C2C6C': 2, '1C2C13N': 2, '2C3C4C': 2, '3C4C5C': 2, '3C4C7C': 2, '4C5C6C': 2, '4C7C8C': 2, '4C7C12C': 2, '4C5C7C': 2, '1C6C13N': 2, '7C8C9C': 2, '7C11C12C': 2, '7C8C12C': 2, '8C9C10C': 2, '9C10C11C': 2, '9C10C14N': 2, '10C11C12C': 2, '10C14N21C': 1, '10C14N37C': 1, '10C11C14N': 2, '13N15C16C': 2, '13N15C20C': 1, '13N27C28C': 1, '13N27C32C': 1, '14N21C22C': 1, '14N21C26C': 1, '14N37C38C': 1,

In [54]:
names

(['1C2C3C',
  '2C3C4C',
  '3C4C5C',
  '3C4C6C',
  '4C5C6C',
  '1C2C3C',
  '2C3C4C',
  '3C4C5C',
  '3C4C6C',
  '4C5C7C',
  '4C5C6C',
  '1C2C3C',
  '1C2C6C',
  '2C3C4C',
  '2C3C5C',
  '2C3C6C',
  '3C4C5C',
  '1C2C3C',
  '1C5C6C',
  '1C2C6C',
  '2C3C4C',
  '3C4C5C',
  '3C4C7C',
  '4C5C6C',
  '4C7C8C',
  '4C5C7C',
  '1C2C3C',
  '1C5C6C',
  '1C2C6C',
  '2C3C4C',
  '3C4C5C',
  '3C4C7C',
  '4C5C6C',
  '4C7C8C',
  '4C7C9C',
  '4C5C7C',
  '7C8C9C',
  '1C2C3C',
  '1C5C6C',
  '1C2C6C',
  '2C3C4C',
  '3C4C5C',
  '3C4C7C',
  '4C5C6C',
  '4C7C8C',
  '4C5C7C',
  '7C8C9C',
  '8C9C10C',
  '9C10C11C',
  '10C11C12C',
  '1C2C3C',
  '1C5C6C',
  '1C2C6C',
  '2C3C4C',
  '3C4C5C',
  '3C4C7C',
  '4C5C6C',
  '4C5C7C',
  '1C2C3C',
  '1C5C6C',
  '1C2C6C',
  '2C3C4C',
  '3C4C5C',
  '3C4C7C',
  '4C5C6C',
  '4C7C8C',
  '4C7C9C',
  '4C5C7C',
  '7C9C10C',
  '7C8C9C',
  '1C2C3C',
  '1C5C6C',
  '1C2C6C',
  '2C3C4C',
  '3C4C5C',
  '3C4C7C',
  '4C5C6C',
  '4C7C8C',
  '4C5C7C',
  '7C8C9C',
  '8C9C10C',
  '1C2C3C',
  '1C5C6

In [64]:
def getMatrix(filename, k, keys, high):
    res_matr = np.zeros((high, len(keys)))
    dic = dict.fromkeys(keys, 0)
    print(dic)
    with open(filename, "r") as f:
        for i in f:
            mol_atoms = []
            num_elems = int(i.split()[1])
            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                line = next(f) #Берём i линию в молекуле
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "000":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, dic)
            print(res)

In [65]:
getMatrix(filename, 3, names[0], names[1])

{'1C2C3C': 0, '2C3C4C': 0, '3C4C5C': 0, '3C4C6C': 0, '4C5C6C': 0, '4C5C7C': 0, '1C2C6C': 0, '2C3C5C': 0, '2C3C6C': 0, '1C5C6C': 0, '3C4C7C': 0, '4C7C8C': 0, '4C7C9C': 0, '7C8C9C': 0, '8C9C10C': 0, '9C10C11C': 0, '10C11C12C': 0, '7C9C10C': 0, '5C6C7C': 0, '1C2C4C': 0, '1C2C5C': 0, '3C5C6C': 0, '6C7C8C': 0, '1C2C7C': 0, '4C7C10C': 0, '7C8C10C': 0, '1C2C8C': 0, '2C3C8C': 0, '1C6C7C': 0, '4C5C13C': 0, '5C13C14C': 0, '5C13C18C': 0, '5C6C13C': 0, '6C7C12C': 0, '7C11C12C': 0, '7C8C12C': 0, '13C14C15C': 0, '13C17C18C': 0, '13C18C19C': 0, '13C14C18C': 0, '14C15C16C': 0, '15C16C17C': 0, '16C17C18C': 0, '17C18C19C': 0, '18C19C20C': 0, '18C19C24C': 0, '19C20C21C': 0, '19C23C24C': 0, '19C20C24C': 0, '20C21C22C': 0, '21C22C23C': 0, '22C23C24C': 0, '1C2C3O': 0, '1C3C4C': 0, '1C2O3C': 0, '1C3C6O': 0, '3C4C5O': 0, '3C4C6O': 0, '1C3C4O': 0, '1C3C5C': 0, '3C4O5C': 0, '1C2C7O': 0, '1C6C7O': 0, '2C3C7O': 0, '2C8C9O': 0, '2C8C10O': 0, '3C4C7O': 0, '8C10O11C': 0, '8C9O10O': 0, '10O11C12C': 0, '10O11C16C': 0,

TypeError: unsupported operand type(s) for +=: 'NoneType' and 'int'

In [ ]:
names[1]